## 📄 Project Documentation: Scientific Image Forgery Detection (RLUC-SIFD)

This notebook implements a semantic segmentation model using a **U-Net architecture** combined with **Error Level Analysis (ELA)** features for detecting and segmenting copy-move forgeries in scientific images. The solution is specifically tailored for the Kaggle $\text{Recod.ai/LUC}$ SIFD competition.

### 1. Configuration & Paths

| Parameter | Value | Description |
| :--- | :--- | :--- |
| $\text{TARGET\_SIZE}$ | $256$ | Standard resolution for image and mask resizing before model input. |
| $\text{DEVICE}$ | `'cuda'` / `'cpu'` | Execution device, prioritizing GPU. |
| $\text{BATCH\_SIZE}$ | $8$ | Batch size for training and inference. |
| $\text{EPOCHS}$ | $3$ | Number of training epochs (reduced for quick iteration). |
| $\text{LEARNING\_RATE}$ | $1e-4$ | Initial learning rate for the Adam optimizer. |
| $\text{FIXED\_THRESHOLD}$ | $0.45$ | Probability threshold applied to U-Net output during inference. |
| $\text{MIN\_FORGERY\_AREA}$ | $32$ | Minimum pixel area for a predicted forgery segment to be kept. |
| $\text{TRAIN\_ROOT}$ | `/kaggle/input/.../train_images` | Path to the training image directories. |
| $\text{MASK\_ROOT}$ | `/kaggle/input/.../train_masks` | Path to the ground-truth binary mask files ($\text{.npy}$). |
| $\text{MODEL\_SAVE\_PATH}$ | `/tmp/model_new_scratch.pth` | Location to save the best-performing model weights. |

---

### 2. Utility and Feature Engineering

#### $2.1.$ `compute_ela(img_path, quality=95, scale=10)`

This function generates the **Error Level Analysis (ELA)** feature map. ELA highlights areas in an image that have a different compression history (often due to forgery, copy-move, or re-saving).

* **Process:**
    1.  Loads the image and resizes it to $\text{TARGET\_SIZE} \times \text{TARGET\_SIZE}$.
    2.  Compresses the resized image as a JPEG with a fixed quality ($\text{quality}=95$).
    3.  Calculates the **absolute difference (error)** between the original resized image and the re-compressed image.
    4.  The mean of the absolute error across the RGB channels is taken, and this $2\text{D}$ error map is scaled by **$10$** ($\text{scale}=10$).
* **Output:** A $2\text{D}$ array ($\text{float32}$) of shape $(\text{TARGET\_SIZE}, \text{TARGET\_SIZE})$.

---

### 3. Model Architecture

#### $3.1.$ `UNet(in_channels=4, num_classes=1)`

A standard **U-Net** architecture adapted for this specific task.

* **Input Channels:** $\text{in\_channels}=4$. This accepts the combined $\text{RGB}$ (3 channels) and $\text{ELA}$ (1 channel) input tensor.
* **Encoder:** Consists of $\text{Conv-ReLU-Dropout-Conv-ReLU}$ blocks and $\text{MaxPool2d}$ for downsampling.
* **Decoder:** Uses $\text{ConvTranspose2d}$ ($\text{upconv}$) for upsampling, followed by concatenation ($\text{torch.cat}$) with the corresponding feature maps from the encoder ($\text{skip connections}$).
* **Output Layer:** A final $\text{Conv2d}$ followed by a **$\text{Sigmoid}$ activation** function, producing a probability map for the forgery mask.

---

### 4. Loss Functions

#### $4.1.$ `DiceLoss(smooth=1.0)`

Implements the **Dice Loss**, a common metric for highly imbalanced segmentation tasks. It measures the overlap between the predicted mask and the ground truth.

$$
\text{Dice Loss} = 1 - \frac{2 \cdot |\text{Pred} \cap \text{Target}| + \text{smooth}}{|\text{Pred}| + |\text{Target}| + \text{smooth}}
$$

#### $4.2.$ `HybridLoss(dice_weight=0.5)`

Combines **Dice Loss** and **Binary Cross-Entropy (BCE) Loss** to provide stable training. BCE is effective for pixel-wise classification, while Dice Loss optimizes for region overlap.

$$
\text{Hybrid Loss} = \text{dice\_weight} \cdot \text{Dice Loss} + (1 - \text{dice\_weight}) \cdot \text{BCE Loss}
$$

* $\text{dice\_weight}$ is set to **$0.5$**, giving equal importance to both terms.

---

### 5. Data Handling

#### $5.1.$ `ForgeryDataset(Dataset)`

A custom PyTorch Dataset class that handles loading, feature generation, and preprocessing for a single sample.

* **Steps:**
    1.  Loads the image ($\text{RGB}$) and the mask ($\text{npy}$).
    2.  Computes the **ELA feature map**.
    3.  Resizes $\text{RGB}$ image and $\text{ELA}$ map to $\text{TARGET\_SIZE}$ ($\text{256}$).
    4.  Resizes the binary mask to $\text{TARGET\_SIZE}$ using $\text{cv2.INTER\_NEAREST}$ to preserve pixel values.
    5.  **Concatenates** the normalized $\text{RGB}$ image and the $\text{ELA}$ feature (after expanding its dimensions) to create the 4-channel input tensor.
    6.  The final input is an $\text{L} \times \text{H} \times \text{W}$ tensor (i.e., $4 \times 256 \times 256$), and the target mask is $1 \times 256 \times 256$.

---

### 6. Inference and Submission

#### $6.1.$ `rle_encode(mask)`

Implements the standard **Run-Length Encoding (RLE)** function required by the competition. If no forgery is detected ($\text{mask.sum()} = 0$), it returns `'authentic'`.

#### $6.2.$ `run_inference_and_segment(unet_model, test_df)`

Handles prediction and post-processing on the test data:

1.  **Prediction:** Processes images in batches, combining $\text{RGB}$ and $\text{ELA}$ features for input.
2.  **Thresholding:** Applies the $\text{FIXED\_THRESHOLD}$ ($\mathbf{0.45}$) to the model's probability output to generate an initial binary mask.
3.  **Post-Processing:** Uses $\text{cv2.connectedComponentsWithStats}$ to identify connected segments. Segments with an area less than $\text{MIN\_FORGERY\_AREA}$ ($\mathbf{32}$) are discarded.
4.  **Resizing:** The cleaned binary mask is resized back to the original image dimensions using $\text{INTER\_NEAREST}$.
5.  **Submission Format:** The final mask is $\text{RLE}$ encoded.
6.  **CSV Output:** The results are written to $\text{submission.csv}$, ensuring the $\text{RLE}$ string is enclosed in brackets (`[RLE_STRING]`) as often required for Kaggle submissions.

## 💾 EDA

In [ ]:
!ls /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/authentic/*.png |wc -l

In [ ]:
!ls /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged/*.png |wc -l 

In [ ]:
!ls /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks/*.npy |wc -l 

In [ ]:
## 📊 Exploratory Data Analysis (EDA)

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm

# --- CONFIGURATION (from the original notebook) ---
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"

print("--- Starting Basic EDA ---")

# 1. Prepare Data List (same logic as in the notebook)
data_list = []
for root, _, files in os.walk(TRAIN_ROOT):
    for f in files:
        valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')
        if f.lower().endswith(valid_extensions):
            if 'forged' in root.lower():
                case_id = os.path.splitext(f)[0]
                img_path = os.path.join(root, f)
                mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")
                data_list.append({
                    'case_id': case_id,
                    'img_path': img_path,
                    'mask_path': mask_path
                })
full_df = pd.DataFrame(data_list)

# Filter for images with existing masks
if not full_df.empty:
    full_df['mask_exists'] = full_df['mask_path'].apply(os.path.exists)
    eda_df = full_df[full_df['mask_exists']].drop(columns=['mask_exists']).reset_index(drop=True)
else:
    eda_df = pd.DataFrame(columns=['case_id', 'img_path', 'mask_path'])


# 2. File and Case Counts
print(f"\nTotal potential images found in TRAIN_ROOT: {len(data_list)}")
print(f"Total valid image/mask pairs for training (Forged Cases): {len(eda_df)}")

# 3. Image and Mask Size Distribution
if not eda_df.empty:
    img_heights, img_widths = [], []
    mask_pixels = [] # Count of non-zero pixels in the mask
    
    print("\nAnalyzing image and mask dimensions/forgery area...")
    for index, row in tqdm(eda_df.iterrows(), total=len(eda_df)):
        try:
            # Read Image
            img = cv2.imread(row['img_path'])
            if img is not None and img.size > 0:
                h, w = img.shape[:2]
                img_heights.append(h)
                img_widths.append(w)

            # Read Mask (npy file)
            mask = np.load(row['mask_path'])
            # Assuming the forgery area is represented by non-zero pixels
            mask_pixels.append(np.sum(mask > 0)) 

        except Exception as e:
            # Handle files that can't be read (e.g., non-image .npy files not handled by cv2.imread)
            pass

    print("\n--- Image Dimension Stats ---")
    print(f"Unique Image Widths: {sorted(list(set(img_widths)))[:5]}{'...' if len(set(img_widths)) > 5 else ''}")
    print(f"Unique Image Heights: {sorted(list(set(img_heights)))[:5]}{'...' if len(set(img_heights)) > 5 else ''}")
    print(f"Mean Image Dimensions (H x W): {np.mean(img_heights):.0f} x {np.mean(img_widths):.0f}")

    # 4. Forgery Area Analysis
    mask_pixels = np.array(mask_pixels)
    forged_cases_with_area = np.sum(mask_pixels > 0)
    total_forgery_area = np.sum(mask_pixels)
    
    print("\n--- Forgery Area Stats ---")
    print(f"Total cases with non-zero forgery area: {forged_cases_with_area} / {len(eda_df)}")
    print(f"Mean Forgery Pixel Count per forged image: {np.mean(mask_pixels[mask_pixels > 0]):.0f} (pixels)")
    print(f"Maximum Forgery Pixel Count: {np.max(mask_pixels)} (pixels)")

    # 5. Visualization: Forgery Area Distribution (First 100 cases for quick view)
    plt.figure(figsize=(12, 5))
    plt.bar(range(min(100, len(mask_pixels))), mask_pixels[:100])
    plt.title('Forgery Pixel Count (First 100 Forged Samples)')
    plt.xlabel('Sample Index')
    plt.ylabel('Forged Pixel Count (Area)')
    plt.show()

else:
    print("🛑 EDA Skipped: The dataframe is empty. Check TRAIN_ROOT and MASK_ROOT paths.")

print("\n--- EDA Complete ---")

## 🚀 Advanced Exploratory Data Analysis (Imbalance & Feature Check)

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION (from the original notebook) ---
TARGET_SIZE = 256
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"

# Replicate compute_ela for feature analysis
def compute_ela(img_path, quality=95, scale=10):
    # ... (omitted for brevity, assume the original function is available)
    # The original notebook's ELA function is used here.
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}.jpg" # Simplified temp_path
    try:
        # Use a consistent quality setting (95)
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality]) 
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale # Scale by 10 as in the notebook
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR).astype(np.float32)

# Load the filtered DataFrame (assuming the prior EDA cell's 'eda_df' is available or recreate it)
data_list = []
for root, _, files in os.walk(TRAIN_ROOT):
    for f in files:
        valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')
        if f.lower().endswith(valid_extensions) and 'forged' in root.lower():
            case_id = os.path.splitext(f)[0]
            mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")
            if os.path.exists(mask_path):
                data_list.append({'img_path': os.path.join(root, f), 'mask_path': mask_path})
eda_df = pd.DataFrame(data_list)

print("--- Starting Advanced EDA (Imbalance & Feature Check) ---")

if eda_df.empty:
    print("🛑 EDA Skipped: Data frame is empty.")
else:
    total_pixels = 0
    forgery_pixels = 0
    ela_values, rgb_means = [], []

    # Process only the first 50 images to speed up ELA computation for EDA
    for index, row in tqdm(eda_df.head(50).iterrows(), total=len(eda_df.head(50)), desc="Processing samples"):
        try:
            # 1. Image and Mask Load
            rgb_image = cv2.cvtColor(cv2.imread(row['img_path']), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0: continue
                
            mask = np.load(row['mask_path'])
            if mask.ndim > 2: mask = mask[:, :, 0]
            
            # 2. Imbalance Check (Use original sizes for best estimate)
            h, w = rgb_image.shape[:2]
            total_pixels += h * w
            forgery_pixels += np.sum(mask > 0)
            
            # 3. ELA Feature Check (Use 256x256 resized data)
            ela_feature = compute_ela(row['img_path'])
            ela_values.extend(ela_feature.flatten())
            
            # RGB feature check (resize/normalize similar to training)
            rgb_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE)) / 255.0
            rgb_means.extend(rgb_resized.mean(axis=2).flatten())

        except Exception as e:
            # print(f"Warning: Could not process {row['img_path']}: {e}")
            continue

    # --- Analysis 1: Imbalance Ratio ---
    if total_pixels > 0:
        imbalance_ratio = (forgery_pixels / total_pixels) * 100
        print(f"\n--- Imbalance Ratio (Forged Pixels) ---")
        print(f"Total Pixels Sampled: {total_pixels:,}")
        print(f"Forged Pixels Sampled: {forgery_pixels:,}")
        print(f"Forgery Imbalance Ratio: **{imbalance_ratio:.2f}%** (Positive Class)")
    
    # --- Analysis 2: ELA Feature Distribution vs. RGB ---
    if ela_values:
        ela_values = np.array(ela_values)
        rgb_means = np.array(rgb_means)

        print(f"\n--- ELA Feature Distribution (Scaled by 10) ---")
        print(f"ELA Feature Mean: {np.mean(ela_values):.4f}")
        print(f"ELA Feature Std Dev: {np.std(ela_values):.4f}")
        print(f"RGB Mean (Normalized): {np.mean(rgb_means):.4f}")

        plt.figure(figsize=(12, 5))
        plt.hist(ela_values, bins=50, alpha=0.6, label='ELA Feature (Scaled)', color='red')
        plt.title('Distribution of ELA Feature Values')
        plt.xlabel('ELA Value (0 to ~2550)')
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()
        
        # This histogram helps visualize if ELA is predominantly zero or clustered.

print("\n--- Advanced EDA Complete ---")

## 🚀 File image test

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

# Define the file path
TEST_IMAGE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images/45.png"

print(f"Attempting to load image: {TEST_IMAGE_PATH}")

if not os.path.exists(TEST_IMAGE_PATH):
    print("🛑 ERROR: The file path was not found. Please ensure the Kaggle competition data is mounted correctly.")
else:
    # Load the image using OpenCV (loads as BGR)
    img = cv2.imread(TEST_IMAGE_PATH)
    
    if img is None:
        print("🛑 ERROR: Could not read the image file.")
    else:
        # Convert the image from BGR (OpenCV default) to RGB (Matplotlib default)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Plot the image
        plt.figure(figsize=(10, 8))
        plt.imshow(img_rgb)
        plt.title(f"Test Image 45 (Dimensions: {img.shape[0]}x{img.shape[1]})")
        plt.axis('off') # Hide axes for a cleaner image view
        plt.show()

## 💾 Final Corrected Training Code

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
from warnings import filterwarnings

# Suppress the specific UserWarning from the LR scheduler
warnings.filterwarnings("ignore", category=UserWarning, module="torch.optim.lr_scheduler")
filterwarnings('ignore')

# --- CONFIGURATION ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
EPOCHS = 1 
LEARNING_RATE = 1e-4

# --- PATHS (CORRECTED FOR KAGGLEHUB CACHE) ---
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"

# --- UTILITY FUNCTIONS ---
def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3:
                img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2:
                img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception:
            return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE),
                      interpolation=cv2.INTER_LINEAR).astype(np.float32)

# --- LOSS FUNCTIONS (FOCAL + DICE) ---
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, pred, target):
        pred = pred.contiguous().view(-1)
        target = target.contiguous().view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        inputs = inputs.contiguous().view(-1)
        targets = targets.contiguous().view(-1)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='none')
        BCE_EXP = torch.exp(-BCE)
        Focal = self.alpha * (1-BCE_EXP)**self.gamma * BCE
        return Focal.mean()

class HybridFocalLoss(nn.Module):
    def __init__(self, dice_weight=0.8):
        super(HybridFocalLoss, self).__init__()
        self.dice_loss = DiceLoss() 
        self.focal_loss = FocalLoss()
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        focal = self.focal_loss(pred, target)
        return self.dice_weight * dice + (1 - self.dice_weight) * focal


# U-Net architecture (Unchanged)
class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

class ForgeryDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['img_path']
        mask_path = row['mask_path']

        # --- Load Image ---
        rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        if rgb_image is None or rgb_image.size == 0:
            try:
                img_data = np.load(img_path)
                if img_data.ndim == 3:
                    rgb_image = img_data
                elif img_data.ndim == 2:
                    rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)
            except Exception as e:
                raise RuntimeError(f"Failed to load image from {img_path}: {e}")

        # --- Load Mask ---
        try:
            mask = np.load(mask_path)
            if mask.ndim > 2:
                mask = mask[:, :, 0]
        except Exception as e:
            raise RuntimeError(f"Failed to load mask from {mask_path}: {e}")

        ela_feature_2d = compute_ela(img_path)

        # Resize all features
        rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
        ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))

        # Use INTER_NEAREST for binary mask resizing
        mask_resized = cv2.resize(mask.astype(np.uint8), (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_NEAREST)

        # Stack RGB (3) and ELA (1) for a 4-channel input
        ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
        stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

        # Convert to PyTorch tensors and normalize
        image = torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
        mask = torch.tensor(mask_resized / 255.0, dtype=torch.float32).unsqueeze(0)

        return image, mask

def train_model(model, train_loader, val_loader, epochs=EPOCHS, save_path=MODEL_SAVE_PATH):
    # Set the new loss function
    criterion = HybridFocalLoss(dice_weight=0.8) 
    
    # CRITICAL CHANGE: Switched to AdamW with Weight Decay to break flat minima
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2) 

    # Scheduler with patience=2 (unchanged)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_val_loss = float('inf')

    model.to(DEVICE)
    print(f"Starting training on {DEVICE} for {epochs} epochs...")

    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0

        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()

        avg_train_loss = train_loss_sum / len(train_loader)

        # Validation Phase
        model.eval()
        val_loss_sum = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss_sum += loss.item()

        avg_val_loss = val_loss_sum / len(val_loader)

        # Scheduler Step
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved successfully to {save_path}. (New best Val Loss: {best_val_loss:.4f})\n")

        current_lr = optimizer.param_groups[0]['lr']
        print(f"Current Learning Rate: {current_lr:.6f}")


# --- FUNCTIONS FOR THRESHOLD OPTIMIZATION (Unchanged) ---
def calculate_dice_coefficient(pred, target, smooth=1.0):
    """Calculates the Dice Similarity Coefficient (DSC) for post-processing tuning."""
    pred = pred.contiguous().view(-1)
    target = target.contiguous().view(-1)
    intersection = (pred * target).sum()
    
    # Dice Coefficient Formula: (2*TP) / (2*TP + FP + FN)
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return dice.item()

def find_optimal_threshold(model, val_loader, device, threshold_range=np.arange(0.30, 0.75, 0.05)):
    """
    Finds the threshold that maximizes the average Dice Coefficient on the validation set.
    """
    model.to(device)
    model.eval()
    
    best_dice = -1.0
    best_threshold = 0.50
    
    print("\n--- Finding Optimal Threshold on Validation Set ---")
    
    # Iterate through each candidate threshold
    for threshold in threshold_range:
        total_dice = 0.0
        
        with torch.no_grad():
            # TQDM wrapper for the val_loader
            for inputs, targets in tqdm(val_loader, desc=f"Evaluating T={threshold:.2f}"):
                inputs, targets = inputs.to(device), targets.to(device)
                
                # 1. Get raw probability output from the model
                outputs = model(inputs)
                
                # 2. Apply current threshold to create a binary prediction mask
                predicted_mask = (outputs > threshold).float()
                
                # 3. Calculate Dice Coefficient for the batch
                batch_dice = calculate_dice_coefficient(predicted_mask, targets)
                total_dice += batch_dice

        avg_dice = total_dice / len(val_loader)
        
        if avg_dice > best_dice:
            best_dice = avg_dice
            best_threshold = threshold
            
        print(f"Threshold: {threshold:.2f} | Avg. Val Dice: {avg_dice:.4f}")

    print(f"\n✅ Optimal Threshold found: {best_threshold:.2f} with Dice: {best_dice:.4f}")
    return best_threshold

# --- MAIN EXECUTION BLOCK ---
if __name__ == '__main__':

    print("Preparing training data paths...\n")

    data_list = []

    # Recursively walk through the TRAIN_ROOT to find all image files
    for root, _, files in os.walk(TRAIN_ROOT):
        for f in files:
            valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')

            if f.lower().endswith(valid_extensions):
                # Only process files in the 'forged' subdirectory, as only they have masks
                if 'forged' in root.lower():
                    case_id = os.path.splitext(f)[0]
                    img_path = os.path.join(root, f)

                    # Use .npy for the mask extension
                    mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")

                    data_list.append({
                        'case_id': case_id,
                        'img_path': img_path,
                        'mask_path': mask_path
                    })

    if not data_list:
        full_df = pd.DataFrame(columns=['case_id', 'img_path', 'mask_path'])
    else:
        full_df = pd.DataFrame(data_list)

    if not full_df.empty:
        # Final check: Keep only images that have a corresponding mask file
        full_df['mask_exists'] = full_df['mask_path'].apply(os.path.exists)
        full_df = full_df[full_df['mask_exists']].drop(columns=['mask_exists']).reset_index(drop=True)

    if full_df.empty:
        print("🛑 FATAL ERROR: No valid image/mask pairs found in the input paths. Cannot train. (Check file extensions/paths again)")
    else:
        print(f"✅ Found {len(full_df)} valid forged samples for training.")

        # Split data
        train_df, val_df = train_test_split(full_df, test_size=0.1, random_state=42)

        # Create DataLoaders
        train_dataset = ForgeryDataset(train_df)
        val_dataset = ForgeryDataset(val_df)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Instantiate model (4 input channels: 3 RGB + 1 ELA)
        model = UNet(in_channels=4)

        # START TRAINING
        train_model(model, train_loader, val_loader)

        print("\n✅ TRAINING COMPLETE.")

        # --- HYPERPARAMETER TUNING: FIND OPTIMAL THRESHOLD ---
        try:
            # Load the best saved model state before running the threshold search
            model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))

            # Run the optimization function
            optimal_threshold = find_optimal_threshold(model, val_loader, DEVICE)

            # Inform the user to update Cell 6
            print(f"\n📢 ACTION REQUIRED: Please use {optimal_threshold:.2f} as the FIXED_THRESHOLD in your final inference code (Cell 6).")
            
        except FileNotFoundError:
            print(f"🛑 ERROR: Trained model not found at {MODEL_SAVE_PATH}. Cannot perform threshold tuning.")

## 💾 Final Corrected Inference Code (Robust Settings)


In [ ]:
!mkdir -p /kaggle/working/test_images

In [ ]:
!cp -pr /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged/10.png /kaggle/working/test_images/

In [ ]:
!cp /kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/authentic/10.png

In [ ]:
!ls /kaggle/working/test_images/

In [ ]:
del TEST_IMAGE_ROOT

In [ ]:
!cat /kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import time
import csv
import warnings
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION & PATHS (ROBUST SETTINGS) ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8

# MODIFIED PARAMETER: Using the optimal threshold calculated from Cell 5
FIXED_THRESHOLD = 0.30 

# ROBUST: Moderate filter to remove noise while keeping small artifacts
MIN_FORGERY_AREA = 32

# CORRECTED PATHS
#TEST_IMAGE_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images"
TEST_IMAGE_ROOT = "/kaggle/working/test_images"
SAMPLE_SUBMISSION_FILE = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv"

# Model path should point to the successfully trained model
model_path = "/tmp/model_new_scratch.pth"
OUTPUT_FILENAME = "submission.csv"

# --- UTILITY FUNCTIONS (Unchanged) ---

def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception:
            return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE),
                      interpolation=cv2.INTER_LINEAR).astype(np.float32)

def create_test_df_robust(test_image_root, sample_submission_path):
    master_df = pd.read_csv(sample_submission_path)
    master_df['case_id'] = master_df['case_id'].astype(str)
    present_files = {}
    if os.path.exists(test_image_root):
        for root, _, files in os.walk(test_image_root):
            for f in files:
                case_id = os.path.splitext(f)[0]
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')):
                    present_files[case_id] = os.path.join(root, f)

    master_df['img_path'] = master_df['case_id'].map(present_files)
    master_df['img_path'] = master_df['img_path'].fillna('MISSING_FILE')
    return master_df[['case_id', 'img_path']]

def rle_encode(mask):
    if mask.sum() == 0: return "authentic"
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ', '.join(str(x) for x in runs)

class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

# --- INFERENCE FUNCTION WITH POST-PROCESSING ---
def run_inference_and_segment(unet_model, test_df):
    results = []
    unet_model.eval()
    images_to_process = []

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing"):
        case = row['case_id']
        img_path = row['img_path']

        if img_path == 'MISSING_FILE' or img_path == 'NOT_FOUND':
            results.append({'case_id': case, 'annotation': 'authentic'})
            continue

        try:
            rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0:
                img_data = np.load(img_path)
                if img_data.ndim == 3: rgb_image = img_data
                elif img_data.ndim == 2: rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)

            if rgb_image is None or rgb_image.size == 0: raise ValueError(f"Invalid image data for {case}")

            original_shape = rgb_image.shape[:2]
            ela_feature_2d = compute_ela(img_path)

            rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
            stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)
            images_to_process.append((case, original_shape, stacked_input))

            # Process batch
            if len(images_to_process) == BATCH_SIZE or index == len(test_df) - 1:
                if images_to_process:
                    batch_inputs = torch.stack([
                        torch.tensor(img_data.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
                        for _, _, img_data in images_to_process
                    ]).to(DEVICE)

                    with torch.no_grad():
                        outputs = unet_model(batch_inputs).detach().cpu().numpy()

                    for i, output in enumerate(outputs):
                        case_id_out, original_shape_out, _ = images_to_process[i]
                        output_prob = output.squeeze()

                        # --- LOG PROBABILITY HERE ---
                        max_prob = np.max(output_prob)
                        print(f"|--- Case {case_id_out} Max Forgery Probability: {max_prob:.4f} ---|")
                        # ----------------------------

                        # Apply Threshold (0.30)
                        final_mask_resized = (output_prob > FIXED_THRESHOLD).astype(np.uint8)

                        # Minimum Area Filtering (32)
                        clean_mask_resized = np.zeros_like(final_mask_resized)

                        # Find connected components
                        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
                            final_mask_resized, 4, cv2.CV_32S
                        )

                        # Iterate through each component (label 0 is the background)
                        for label in range(1, num_labels):
                            area = stats[label, cv2.CC_STAT_AREA]
                            if area >= MIN_FORGERY_AREA:
                                # Keep segments that meet the minimum size requirement
                                clean_mask_resized[labels == label] = 1

                        # Resize the CLEANED mask back to the original size
                        final_mask = cv2.resize(
                            clean_mask_resized,
                            (original_shape_out[1], original_shape_out[0]),
                            interpolation=cv2.INTER_NEAREST
                        )

                        rle_annotation = rle_encode(final_mask)
                        results.append({'case_id': case_id_out, 'annotation': rle_annotation})

                    images_to_process = [] # Reset batch
        except Exception as e:
            print(f"Error processing case {case}: {e}. Defaulting to authentic.")
            results.append({'case_id': case, 'annotation': 'authentic'})
    return pd.DataFrame(results)

# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":

    print(f"--- Starting inference on {DEVICE} at {pd.Timestamp.now()} ---")

    # 1. Load Model
    model = None
    try:
        model = UNet(in_channels=4).to(DEVICE)
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        model.eval() # Set model to evaluation mode
        print(f"Model loaded successfully from {model_path}")
    except Exception as e:
        print(f"Error loading model from {model_path}. Submitting 'authentic' for all cases. Error: {e}")
        model = None

    # 2. Prepare Data
    test_df = create_test_df_robust(TEST_IMAGE_ROOT, SAMPLE_SUBMISSION_FILE)
    test_df['case_id'] = test_df['case_id'].astype(str)

    # 3. Run Inference
    if model:
        results_df = run_inference_and_segment(model, test_df)
        print(results_df)
    else:
        results_df = test_df[['case_id']].assign(annotation='authentic')

    # 4. Finalize Submission DF
    submission_df = test_df[['case_id']].copy().merge(results_df, on='case_id', how='left')
    submission_df['annotation'] = submission_df['annotation'].fillna('authentic')
    submission_df = submission_df[['case_id', 'annotation']].sort_values('case_id').reset_index(drop=True)

    # 5. Write CSV with Correct RLE Formatting
    with open(OUTPUT_FILENAME, "w", newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['case_id', 'annotation'])

        for _, row in submission_df.iterrows():
            case_id = str(row['case_id'])
            annotation = row['annotation']

            if annotation.lower() == 'authentic':
                 writer.writerow([case_id, annotation])
            else:
                 # Create the full bracketed RLE string
                 full_rle_string = f"[{annotation}]"
                 writer.writerow([case_id, full_rle_string])

    print(f"\n✅ Created {OUTPUT_FILENAME} with {len(submission_df)} rows at {pd.Timestamp.now()}")

## 💾 submission

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm.auto import tqdm
import time
import csv
import warnings
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION & PATHS (Corrected for case 10) ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
FIXED_THRESHOLD = 0.30 # Optimal threshold from validation
MIN_FORGERY_AREA = 32  # Minimum area filter

# The model must be successfully saved during the training phase
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"

# Since you copied 10.png to this folder, we target it here.
TEST_IMAGE_ROOT = "/kaggle/working/test_images"
# Create a dummy submission file for case_id '10' only
CUSTOM_SUB_PATH = "/tmp/custom_sample_submission_10.csv"
OUTPUT_FILENAME = "submission_case10.csv"

# --- UTILITY FUNCTIONS ---
def compute_ela(img_path, quality=95, scale=10):
    """Generates the Error Level Analysis (ELA) feature map."""
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception:
            return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE),
                      interpolation=cv2.INTER_LINEAR).astype(np.float32)

def create_test_df_robust(test_image_root, sample_submission_path):
    """Creates a DataFrame mapping case_ids to image paths."""
    master_df = pd.read_csv(sample_submission_path)
    master_df['case_id'] = master_df['case_id'].astype(str)
    present_files = {}
    if os.path.exists(test_image_root):
        for root, _, files in os.walk(test_image_root):
            for f in files:
                case_id = os.path.splitext(f)[0]
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')):
                    present_files[case_id] = os.path.join(root, f)

    master_df['img_path'] = master_df['case_id'].map(present_files)
    master_df['img_path'] = master_df['img_path'].fillna('MISSING_FILE')
    return master_df[['case_id', 'img_path']]

def rle_encode(mask):
    """Encodes a binary mask using Run-Length Encoding. Returns 'authentic' if empty."""
    if mask.sum() == 0: return "authentic"
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ', '.join(str(x) for x in runs)

# --- U-NET ARCHITECTURE (copied from notebook) ---
class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

# --- INFERENCE FUNCTION WITH POST-PROCESSING ---
def run_inference_and_segment(unet_model, test_df, output_file):
    results = []
    unet_model.eval()
    images_to_process = []
    case_info = []

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing"):
        case = row['case_id']
        img_path = row['img_path']

        if img_path == 'MISSING_FILE' or img_path == 'NOT_FOUND':
            results.append({'case_id': case, 'annotation': 'authentic'})
            continue

        try:
            rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0:
                # Add numpy file handling for robustness
                img_data = np.load(img_path)
                if img_data.ndim == 3: rgb_image = img_data
                elif img_data.ndim == 2: rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)

            if rgb_image is None or rgb_image.size == 0: raise ValueError(f"Invalid image data for {case}")

            original_shape = rgb_image.shape[:2]
            ela_feature_2d = compute_ela(img_path)

            # Preprocessing
            rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
            stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

            images_to_process.append(
                torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
            )
            case_info.append((case, original_shape))

            # Process batch
            if len(images_to_process) == BATCH_SIZE or index == len(test_df) - 1:
                if images_to_process:
                    batch_inputs = torch.stack(images_to_process).to(DEVICE)

                    with torch.no_grad():
                        outputs = unet_model(batch_inputs).detach().cpu().numpy()

                    for i, output in enumerate(outputs):
                        case_id_out, original_shape_out = case_info[i]
                        output_prob = output.squeeze()

                        # Thresholding
                        final_mask_resized = (output_prob > FIXED_THRESHOLD).astype(np.uint8)

                        # Minimum Area Filtering (Post-processing)
                        clean_mask_resized = np.zeros_like(final_mask_resized)
                        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
                            final_mask_resized, 4, cv2.CV_32S
                        )
                        for label in range(1, num_labels):
                            area = stats[label, cv2.CC_STAT_AREA]
                            if area >= MIN_FORGERY_AREA:
                                clean_mask_resized[labels == label] = 1

                        # Resize back to original dimensions
                        final_mask = cv2.resize(
                            clean_mask_resized,
                            (original_shape_out[1], original_shape_out[0]),
                            interpolation=cv2.INTER_NEAREST
                        )

                        rle_annotation = rle_encode(final_mask)
                        results.append({'case_id': case_id_out, 'annotation': rle_annotation})

                    images_to_process = []
                    case_info = []

        except Exception as e:
            print(f"Error processing case {case}: {e}. Defaulting to authentic.")
            results.append({'case_id': case, 'annotation': 'authentic'})
    
    # Finalize Submission DF and write CSV
    results_df = pd.DataFrame(results)
    submission_df = test_df[['case_id']].copy().merge(results_df, on='case_id', how='left')
    submission_df['annotation'] = submission_df['annotation'].fillna('authentic')
    submission_df = submission_df[['case_id', 'annotation']].sort_values('case_id').reset_index(drop=True)

    with open(output_file, "w", newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['case_id', 'annotation'])
        for _, row in submission_df.iterrows():
            case_id = str(row['case_id'])
            annotation = row['annotation']
            if annotation.lower() == 'authentic':
                 writer.writerow([case_id, annotation])
            else:
                 full_rle_string = f"[{annotation}]"
                 writer.writerow([case_id, full_rle_string])
    
    return submission_df

# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":
    
    # 1. SETUP: Create Custom Test Submission for case_id '10'
    custom_sub_df = pd.DataFrame({'case_id': ['10'], 'annotation': ['authentic']})
    custom_sub_df.to_csv(CUSTOM_SUB_PATH, index=False)
    
    print(f"--- Starting Inference for case 10 on {DEVICE} ---")

    # 2. Load Model
    model = None
    try:
        model = UNet(in_channels=4).to(DEVICE)
        # Attempt to load the best-performing weights
        model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
        model.eval()
        print(f"Model loaded successfully from {MODEL_SAVE_PATH}")
    except Exception as e:
        print(f"🛑 ERROR: Model weights not found at {MODEL_SAVE_PATH}. Cannot run inference.")
        print("Defaulting to 'authentic' for all test cases.")
        model = None
    
    # 3. Prepare Data
    test_df = create_test_df_robust(TEST_IMAGE_ROOT, CUSTOM_SUB_PATH)
    
    # 4. Run Inference
    if model and not test_df.empty:
        results_df = run_inference_and_segment(model, test_df, OUTPUT_FILENAME)
        
        print(f"\n✅ Created {OUTPUT_FILENAME} with the prediction for case 10.")
        print("\n--- Resulting Submission File Content ---")
        
        # Print the contents of the generated CSV file
        with open(OUTPUT_FILENAME, 'r') as f:
            print(f.read())
    else:
        # If model failed to load, create a default 'authentic' submission for the case
        submission_df = test_df[['case_id']].assign(annotation='authentic')
        submission_df.to_csv(OUTPUT_FILENAME, index=False)
        print(f"🛑 Failed to run model. Generated a default 'authentic' submission for case 10 in {OUTPUT_FILENAME}.")

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm.auto import tqdm
import time
import csv
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION OVERRIDES FOR DEBUGGING ---
# Setting the threshold extremely low and area filter to 1 to capture ANY prediction
FIXED_THRESHOLD = 0.05 
MIN_FORGERY_AREA = 1 

# Paths and other parameters remain the same
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"
TEST_IMAGE_ROOT = "/kaggle/working/test_images"
CUSTOM_SUB_PATH = "/tmp/custom_sample_submission_10.csv"
OUTPUT_FILENAME_DEBUG = "submission_10_DEBUG.csv"

# --- UTILITY & MODEL DEFINITIONS (Required to run, must be available in scope) ---
# (rle_encode, compute_ela, UNet, create_test_df_robust, run_inference_and_segment functions 
# are assumed to be defined in previous notebook cells.)

# --- MAIN EXECUTION BLOCK (Debug Run) ---
if __name__ == "__main__":
    
    # SETUP: Create Custom Test Submission for case_id '10' (re-creates the file)
    custom_sub_df = pd.DataFrame({'case_id': ['10'], 'annotation': ['authentic']})
    custom_sub_df.to_csv(CUSTOM_SUB_PATH, index=False)
    
    print(f"--- Starting DEBUG Inference for case 10 (Threshold={FIXED_THRESHOLD}, MinArea={MIN_FORGERY_AREA}) ---")

    # 1. Load Model
    model = None
    try:
        model = UNet(in_channels=4).to(DEVICE)
        model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
        model.eval()
    except Exception:
        print(f"🛑 ERROR: Model weights not found. Cannot run debugging inference.")
        exit()
    
    # 2. Prepare Data
    test_df = create_test_df_robust(TEST_IMAGE_ROOT, CUSTOM_SUB_PATH)
    
    # 3. Run Inference with Debug Settings
    if model and not test_df.empty:
        # Note: We use the existing run_inference_and_segment function which utilizes the global FIXED_THRESHOLD and MIN_FORGERY_AREA
        results_df = run_inference_and_segment(model, test_df, OUTPUT_FILENAME_DEBUG)
        
        print(f"\n✅ DEBUG output generated in {OUTPUT_FILENAME_DEBUG}.")
        print("\n--- DEBUG Submission File Content ---")
        
        # Print the contents of the generated CSV file
        with open(OUTPUT_FILENAME_DEBUG, 'r') as f:
            print(f.read())

In [ ]:
!cat submission.csv

In [ ]:
def validate_and_print_rle(submission_df):
    """
    Validates RLE output structure and prints debugging info.
    Checks for: 1. Authentic/RLE count. 2. Even number of RLE elements.
    """
    print("\n--- RLE Output Validation Check ---")

    # Analyze the annotations
    authentic_count = submission_df['annotation'].apply(lambda x: x == 'authentic').sum()
    rle_rows = submission_df[submission_df['annotation'] != 'authentic']

    print(f"Total Submissions: {len(submission_df)}")
    print(f"Authentic (No Forgery) Count: {authentic_count}")
    print(f"RLE Annotated (Forged) Count: {len(rle_rows)}")

    # CRITICAL CHECK: RLE strings must always have an even number of elements (start, length, start, length...)
    rle_check = rle_rows['annotation'].apply(lambda x: len(x.split(' ')) % 2 == 0)

    if rle_check.all():
        print(f"✅ RLE Structure: All {len(rle_rows)} RLE strings contain an even number of elements.")
    else:
        # Prints a warning if any RLE string has an odd number of elements (a common error)
        bad_rle_count = len(rle_rows) - rle_check.sum()
        print(f"🛑 RLE ERROR: Found {bad_rle_count} RLE strings with an odd number of elements (Invalid pairing).")

In [ ]:
submission_df = pd.read_csv("submission.csv")
validate_and_print_rle(submission_df)

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm # Use tqdm.auto for notebook compatibility
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
from warnings import filterwarnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.optim.lr_scheduler")
filterwarnings('ignore')

# --- CONFIGURATION (FINAL STABLE FIXES) ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
# FINAL FIX 1: Set max epochs to 10
EPOCHS = 10 
# FINAL FIX 2: Reduced LR for stable convergence
LEARNING_RATE = 1e-5 

# --- PATHS (As per original notebook) ---
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"

# --- UTILITY FUNCTIONS (Unchanged) ---
def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE),
                      interpolation=cv2.INTER_LINEAR).astype(np.float32)


# --- LOSS FUNCTIONS (FIXED) ---
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, pred, target):
        pred = pred.contiguous().view(-1)
        target = target.contiguous().view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        inputs = inputs.contiguous().view(-1)
        targets = targets.contiguous().view(-1)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='none')
        BCE_EXP = torch.exp(-BCE)
        Focal = self.alpha * (1-BCE_EXP)**self.gamma * BCE
        return Focal.mean()

class HybridFocalLoss(nn.Module):
    # FINAL FIX 3: Balance loss weights for stable convergence
    def __init__(self, dice_weight=0.5): 
        super(HybridFocalLoss, self).__init__()
        self.dice_loss = DiceLoss() 
        self.focal_loss = FocalLoss()
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        focal = self.focal_loss(pred, target)
        return self.dice_weight * dice + (1 - self.dice_weight) * focal


# U-Net architecture (Unchanged)
class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

class ForgeryDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['img_path']
        mask_path = row['mask_path']

        # --- Load Image (Robust) ---
        rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        if rgb_image is None or rgb_image.size == 0:
            try:
                img_data = np.load(img_path)
                if img_data.ndim == 3: rgb_image = img_data
                elif img_data.ndim == 2: rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)
            except Exception as e:
                raise RuntimeError(f"Failed to load image from {img_path}: {e}")

        # --- Load Mask (Robust) ---
        try:
            mask = np.load(mask_path)
            if mask.ndim > 2: mask = mask[:, :, 0]
        except Exception as e:
            raise RuntimeError(f"Failed to load mask from {mask_path}: {e}")

        ela_feature_2d = compute_ela(img_path)

        # Resize all features
        rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
        ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
        mask_resized = cv2.resize(mask.astype(np.uint8), (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_NEAREST)

        # Stack RGB (3) and ELA (1)
        ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
        stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

        # Convert to PyTorch tensors and normalize
        image = torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
        mask = torch.tensor(mask_resized / 255.0, dtype=torch.float32).unsqueeze(0)

        return image, mask

def train_model(model, train_loader, val_loader, epochs=EPOCHS, save_path=MODEL_SAVE_PATH):
    criterion = HybridFocalLoss() 
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2) 
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_val_loss = float('inf')
    
    # --- EARLY STOPPING PARAMETERS ---
    PATIENCE = 5                        # Stop if no improvement after 5 epochs
    epochs_no_improve = 0
    MIN_DELTA = 1e-5                    # Minimum improvement required to reset patience
    # ---------------------------------

    model.to(DEVICE)
    print(f"Starting stabilized training on {DEVICE} for max {epochs} epochs with LR={LEARNING_RATE}...")

    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()

        avg_train_loss = train_loss_sum / len(train_loader)

        # Validation Phase
        model.eval()
        val_loss_sum = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss_sum += loss.item()

        avg_val_loss = val_loss_sum / len(val_loader)

        # Scheduler Step
        scheduler.step(avg_val_loss)

        current_lr = optimizer.param_groups[0]['lr']

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Current LR: {current_lr:.6f}")

        # --- MODEL CHECKPOINTING & EARLY STOPPING LOGIC ---
        if avg_val_loss < best_val_loss - MIN_DELTA:
            # 1. New best loss found: Reset counter and save model
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), save_path)
            print(f"Model saved successfully to {save_path}. (New best Val Loss: {best_val_loss:.4f})\n")
        else:
            # 2. No significant improvement: Increment counter
            epochs_no_improve += 1
            print(f"Validation loss did not significantly improve. Patience left: {PATIENCE - epochs_no_improve}\n")

        # 3. Terminate if patience runs out
        if epochs_no_improve >= PATIENCE:
            print(f"🛑 EARLY STOPPING triggered after {epoch+1} epochs (patience={PATIENCE}). Stopping training.")
            break
            
    print("Training loop finished.")


# --- FUNCTIONS FOR THRESHOLD OPTIMIZATION (Unchanged) ---
def calculate_dice_coefficient(pred, target, smooth=1.0):
    pred = pred.contiguous().view(-1)
    target = target.contiguous().view(-1)
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return dice.item()

def find_optimal_threshold(model, val_loader, device, threshold_range=np.arange(0.05, 0.75, 0.05)):
    model.to(device)
    model.eval()
    
    best_dice = -1.0
    best_threshold = 0.50
    
    print("\n--- Finding Optimal Threshold on Validation Set (Expanded Range) ---")
    
    for threshold in threshold_range:
        total_dice = 0.0
        
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc=f"Evaluating T={threshold:.2f}"):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                predicted_mask = (outputs > threshold).float()
                batch_dice = calculate_dice_coefficient(predicted_mask, targets)
                total_dice += batch_dice

        avg_dice = total_dice / len(val_loader)
        
        if avg_dice > best_dice:
            best_dice = avg_dice
            best_threshold = threshold
            
        # print(f"Threshold: {threshold:.2f} | Avg. Val Dice: {avg_dice:.4f}") # Omitted for cleaner log

    print(f"\n✅ Optimal Threshold found: {best_threshold:.2f} with Dice: {best_dice:.4f}")
    return best_threshold


# --- MAIN EXECUTION BLOCK (Unchanged) ---
if __name__ == '__main__':

    print("Preparing training data paths...\n")

    data_list = []
    for root, _, files in os.walk(TRAIN_ROOT):
        for f in files:
            valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')
            if f.lower().endswith(valid_extensions):
                if 'forged' in root.lower():
                    case_id = os.path.splitext(f)[0]
                    img_path = os.path.join(root, f)
                    mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")
                    data_list.append({
                        'case_id': case_id,
                        'img_path': img_path,
                        'mask_path': mask_path
                    })

    if not data_list:
        full_df = pd.DataFrame(columns=['case_id', 'img_path', 'mask_path'])
    else:
        full_df = pd.DataFrame(data_list)

    if not full_df.empty:
        full_df['mask_exists'] = full_df['mask_path'].apply(os.path.exists)
        full_df = full_df[full_df['mask_exists']].drop(columns=['mask_exists']).reset_index(drop=True)

    if full_df.empty:
        print("🛑 FATAL ERROR: No valid image/mask pairs found. Cannot train.")
    else:
        print(f"✅ Found {len(full_df)} valid forged samples for stabilized training.")

        # Split data
        train_df, val_df = train_test_split(full_df, test_size=0.1, random_state=42)

        # Create DataLoaders
        train_dataset = ForgeryDataset(train_df)
        val_dataset = ForgeryDataset(val_df)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Instantiate model
        model = UNet(in_channels=4)

        # START STABILIZED TRAINING
        train_model(model, train_loader, val_loader)

        print("\n✅ STABILIZED TRAINING COMPLETE.")

        # --- HYPERPARAMETER TUNING: FIND OPTIMAL THRESHOLD ---
        try:
            model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
            optimal_threshold = find_optimal_threshold(model, val_loader, DEVICE)
            print(f"\n📢 ACTION REQUIRED: Please use {optimal_threshold:.2f} as the FIXED_THRESHOLD in your final inference code.")
            
        except FileNotFoundError:
            print(f"🛑 ERROR: Trained model not found at {MODEL_SAVE_PATH}. Cannot perform threshold tuning.")

## inference

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm.auto import tqdm
import time
import csv
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION & PATHS (Using final, stable settings) ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8

# CRITICAL: This value MUST be updated after the 15-epoch training run is complete.
# Use the result from the 'Optimal Threshold found' message.
FIXED_THRESHOLD = 0.45 # <<<--- UPDATE THIS AFTER TRAINING IS COMPLETE

MIN_FORGERY_AREA = 32 # Retaining robust post-processing filter

# Path to the actual Kaggle test set folder (for submission)
TEST_IMAGE_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images"
SAMPLE_SUBMISSION_FILE = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv"
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"
OUTPUT_FILENAME = "submission.csv"

# --- UTILITY FUNCTIONS (Must be in scope) ---
def compute_ela(img_path, quality=95, scale=10):
    # ... (function body as defined in original Cell 12) ...
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
    if img is None or img.size == 0: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR).astype(np.float32)

def create_test_df_robust(test_image_root, sample_submission_path):
    master_df = pd.read_csv(sample_submission_path)
    master_df['case_id'] = master_df['case_id'].astype(str)
    present_files = {}
    if os.path.exists(test_image_root):
        for root, _, files in os.walk(test_image_root):
            for f in files:
                case_id = os.path.splitext(f)[0]
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')):
                    present_files[case_id] = os.path.join(root, f)
    master_df['img_path'] = master_df['case_id'].map(present_files)
    master_df['img_path'] = master_df['img_path'].fillna('MISSING_FILE')
    return master_df[['case_id', 'img_path']]

def rle_encode(mask):
    if mask.sum() == 0: return "authentic"
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ', '.join(str(x) for x in runs)

# --- U-NET ARCHITECTURE (Must be in scope) ---
class UNet(nn.Module):
    # ... (UNet definition body from Cell 12) ...
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

# --- INFERENCE FUNCTION WITH POST-PROCESSING ---
def run_inference_and_segment(unet_model, test_df):
    results = []
    unet_model.eval()
    images_to_process = []
    case_info = []

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing"):
        case = row['case_id']
        img_path = row['img_path']

        if img_path == 'MISSING_FILE' or img_path == 'NOT_FOUND':
            results.append({'case_id': case, 'annotation': 'authentic'})
            continue

        try:
            rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0:
                img_data = np.load(img_path)
                if img_data.ndim == 3: rgb_image = img_data
                elif img_data.ndim == 2: rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)

            if rgb_image is None or rgb_image.size == 0: raise ValueError(f"Invalid image data for {case}")

            original_shape = rgb_image.shape[:2]
            ela_feature_2d = compute_ela(img_path)

            rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
            stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

            images_to_process.append(
                torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
            )
            case_info.append((case, original_shape))

            if len(images_to_process) == BATCH_SIZE or index == len(test_df) - 1:
                if images_to_process:
                    batch_inputs = torch.stack(images_to_process).to(DEVICE)

                    with torch.no_grad():
                        outputs = unet_model(batch_inputs).detach().cpu().numpy()

                    for i, output in enumerate(outputs):
                        case_id_out, original_shape_out = case_info[i]
                        output_prob = output.squeeze()
                        
                        # Apply the newly found optimal FIXED_THRESHOLD
                        final_mask_resized = (output_prob > FIXED_THRESHOLD).astype(np.uint8)

                        # Minimum Area Filtering
                        clean_mask_resized = np.zeros_like(final_mask_resized)
                        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
                            final_mask_resized, 4, cv2.CV_32S
                        )
                        for label in range(1, num_labels):
                            area = stats[label, cv2.CC_STAT_AREA]
                            if area >= MIN_FORGERY_AREA:
                                clean_mask_resized[labels == label] = 1

                        # Resize back to original dimensions
                        final_mask = cv2.resize(
                            clean_mask_resized,
                            (original_shape_out[1], original_shape_out[0]),
                            interpolation=cv2.INTER_NEAREST
                        )

                        rle_annotation = rle_encode(final_mask)
                        results.append({'case_id': case_id_out, 'annotation': rle_annotation})

                    images_to_process = []
                    case_info = []

        except Exception as e:
            print(f"Error processing case {case}: {e}. Defaulting to authentic.")
            results.append({'case_id': case, 'annotation': 'authentic'})
    
    return pd.DataFrame(results)

# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":

    print(f"--- Starting Final Inference on {DEVICE} with Threshold={FIXED_THRESHOLD} ---")

    # 1. Load Model (The new, stable model)
    model = None
    try:
        model = UNet(in_channels=4).to(DEVICE)
        model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
        model.eval()
        print(f"Model loaded successfully from {MODEL_SAVE_PATH}")
    except Exception as e:
        print(f"🛑 ERROR: Model weights not found at {MODEL_SAVE_PATH}. Cannot run inference.")
        print("Please wait for the training (Cell 12) to fully complete.")
        model = None

    # 2. Prepare Data (Targets the full Kaggle test set)
    test_df = create_test_df_robust(TEST_IMAGE_ROOT, SAMPLE_SUBMISSION_FILE)

    # 3. Run Inference
    if model:
        results_df = run_inference_and_segment(model, test_df)
    else:
        results_df = test_df[['case_id']].assign(annotation='authentic')

    # 4. Finalize Submission DF and write CSV
    submission_df = test_df[['case_id']].copy().merge(results_df, on='case_id', how='left')
    submission_df['annotation'] = submission_df['annotation'].fillna('authentic')
    submission_df = submission_df[['case_id', 'annotation']].sort_values('case_id').reset_index(drop=True)

    with open(OUTPUT_FILENAME, "w", newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['case_id', 'annotation'])
        for _, row in submission_df.iterrows():
            case_id = str(row['case_id'])
            annotation = row['annotation']
            if annotation.lower() == 'authentic':
                 writer.writerow([case_id, annotation])
            else:
                 # Format the RLE string with brackets
                 full_rle_string = f"[{annotation}]"
                 writer.writerow([case_id, full_rle_string])

    print(f"\n✅ Created final submission file: {OUTPUT_FILENAME}")
    print("\n--- Next Step ---")
    print(f"Once training gives a new optimal threshold (e.g., 0.55), update the 'FIXED_THRESHOLD' variable and rerun this cell.")

## experiment test 

In [ ]:
!cat submission.csv

In [ ]:
import numpy as np
import os
import torch
import pandas as pd
import csv
import shutil
import cv2
from tqdm.auto import tqdm
import time
import torch.nn as nn
import torch.nn.functional as F

# --- CONFIGURATION (FINAL EXPERIMENT SETTINGS) ---
# NOTE: These settings MUST override the defaults for this specific test
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"
TEST_WORKING_ROOT = "/kaggle/working/validation_test_final"
TARGET_SIZE = 256
BATCH_SIZE = 8

# CRITICAL FIX 1: Set threshold to the absolute minimum to capture any signal
FIXED_THRESHOLD = 0.05 
# CRITICAL FIX 2: Set minimum area to 1 to ensure NO pixels are filtered out
MIN_FORGERY_AREA = 1 


FIXED_THRESHOLD = 0.45
MIN_FORGERY_AREA = 32

# --- PATHS ---
FORGED_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged/10.png"
AUTHENTIC_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/authentic/10.png"


# --- UTILITY & MODEL DEFINITIONS (Must be defined in the notebook) ---
# Re-define necessary functions/classes to ensure they are available in this cell's scope.

def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    # [Simplified loading logic for brevity, assuming full logic is in notebook]
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR).astype(np.float32)

def rle_encode(mask):
    """Encodes a mask. Returns 'authentic' if empty."""
    if mask.sum() == 0: return "authentic"
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    # Final fix for submission: return space-separated string without external brackets
    return ' '.join(str(x) for x in runs) 

def create_test_df_robust(test_image_root, sample_submission_path):
    master_df = pd.read_csv(sample_submission_path)
    master_df['case_id'] = master_df['case_id'].astype(str)
    present_files = {}
    if os.path.exists(test_image_root):
        for root, _, files in os.walk(test_image_root):
            for f in files:
                case_id = os.path.splitext(f)[0]
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')):
                    present_files[case_id] = os.path.join(root, f)
    master_df['img_path'] = master_df['case_id'].map(present_files)
    master_df['img_path'] = master_df['img_path'].fillna('MISSING_FILE')
    return master_df[['case_id', 'img_path']]

class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        # Ensure block logic is correctly defined and indented
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

def run_inference_and_segment(unet_model, test_df):
    """Executes inference using the CURRENT GLOBAL FIXED_THRESHOLD and MIN_FORGERY_AREA."""
    # Note: Global variables FIXED_THRESHOLD, MIN_FORGERY_AREA, TARGET_SIZE, DEVICE are used directly.
    
    results = []
    unet_model.eval()
    images_to_process = []
    case_info = []

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing"):
        case = row['case_id']
        img_path = row['img_path']
        # ... [Image loading and ELA calculation remains the same] ...
        
        try:
            rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0:
                img_data = np.load(img_path)
                if img_data.ndim == 3: rgb_image = img_data
                elif img_data.ndim == 2: rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)
            if rgb_image is None or rgb_image.size == 0: raise ValueError(f"Invalid image data for {case}")

            original_shape = rgb_image.shape[:2]
            ela_feature_2d = compute_ela(img_path)
            
            # Preprocessing
            rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
            stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

            images_to_process.append(
                torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
            )
            case_info.append((case, original_shape))

            if len(images_to_process) == BATCH_SIZE or index == len(test_df) - 1:
                if images_to_process:
                    batch_inputs = torch.stack(images_to_process).to(DEVICE)

                    with torch.no_grad():
                        outputs = unet_model(batch_inputs).detach().cpu().numpy()

                    for i, output in enumerate(outputs):
                        case_id_out, original_shape_out = case_info[i]
                        output_prob = output.squeeze()
                        
                        final_mask_resized = (output_prob > FIXED_THRESHOLD).astype(np.uint8)

                        # --- CRITICAL AREA FILTERING (Uses MIN_FORGERY_AREA) ---
                        clean_mask_resized = np.zeros_like(final_mask_resized)
                        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
                            final_mask_resized, 4, cv2.CV_32S
                        )
                        for label in range(1, num_labels):
                            area = stats[label, cv2.CC_STAT_AREA]
                            if area >= MIN_FORGERY_AREA: 
                                clean_mask_resized[labels == label] = 1

                        final_mask = cv2.resize(
                            clean_mask_resized,
                            (original_shape_out[1], original_shape_out[0]),
                            interpolation=cv2.INTER_NEAREST
                        )
                        # --- END CRITICAL AREA FILTERING ---

                        rle_annotation = rle_encode(final_mask)
                        results.append({'case_id': case_id_out, 'annotation': rle_annotation})

                    images_to_process = []
                    case_info = []

        except Exception as e:
            # print(f"Error processing case {case}: {e}. Defaulting to authentic.") # Suppressed for cleaner log
            results.append({'case_id': case, 'annotation': 'authentic'})
    
    return pd.DataFrame(results)


# --- MAIN EXPERIMENT EXECUTION ---
if __name__ == "__main__":
    
    # 1. Setup Environment (Clean and copy files)
    if os.path.exists(TEST_WORKING_ROOT):
        shutil.rmtree(TEST_WORKING_ROOT)
    os.makedirs(TEST_WORKING_ROOT)

    shutil.copy(FORGED_PATH, os.path.join(TEST_WORKING_ROOT, "F.png"))
    shutil.copy(AUTHENTIC_PATH, os.path.join(TEST_WORKING_ROOT, "A.png"))

    # Create submission file mapping F -> F.png and A -> A.png
    experiment_data = pd.DataFrame({
        'case_id': ['F', 'A'],
        'annotation': ['authentic', 'authentic']
    })
    EXPERIMENT_SUB_PATH = os.path.join(TEST_WORKING_ROOT, "experiment_sub.csv")
    experiment_data.to_csv(EXPERIMENT_SUB_PATH, index=False)

    print(f"--- Starting FINAL Validation Experiment ---")
    print(f"Testing: Threshold={FIXED_THRESHOLD} and Min Area={MIN_FORGERY_AREA} (Disabled Filter)")

    # 2. Load Model
    try:
        model = UNet(in_channels=4).to(DEVICE)
        model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
        model.eval()
    except Exception as e:
        print(f"🛑 ERROR: Model not loaded. Cannot run validation. Error: {e}")
        exit()

    # 3. Run Inference
    test_df = create_test_df_robust(TEST_WORKING_ROOT, EXPERIMENT_SUB_PATH)
    results_df = run_inference_and_segment(model, test_df)
    results_df = results_df.set_index('case_id')

    # 4. Print Validation Results
    print("\n--- EXPERIMENT RESULTS ---")
    
    forged_result = results_df.loc['F', 'annotation']
    authentic_result = results_df.loc['A', 'annotation']
    
    print(f"FORGED Case (F) Prediction: {forged_result}")
    print(f"AUTHENTIC Case (A) Prediction: {authentic_result}")

    print("\n--- LOGIC CHECK ---")
    
    # Check 1: Positive Control (Forged)
    if forged_result.lower() != 'authentic':
        print("✅ FORGED CHECK PASSED: Model now detects the known forgery (output RLE).")
    else:
        print("🛑 FORGED CHECK FAILED: Model still predicts 'authentic' even with disabled area filter.")

    # Check 2: Negative Control (Authentic)
    if authentic_result.lower() == 'authentic':
        print("✅ AUTHENTIC CHECK PASSED: Model correctly ignores the authentic file.")
    else:
        print("🛑 AUTHENTIC CHECK FAILED: Model predicted forgery (RLE) for a known authentic file.")

In [ ]:
import numpy as np
import os
import torch
import cv2
import pandas as pd
from tqdm.auto import tqdm
import time
import torch.nn as nn
import torch.nn.functional as F

# --- CONFIGURATION (Uses Global Settings) ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
TARGET_SIZE = 256
BATCH_SIZE = 8

# Suspicion threshold: Files with max confidence below this are flagged.
SUSPICION_THRESHOLD = 0.40 
MIN_FORGERY_AREA = 1 # Set to 1 to check raw model confidence, disabling filtering

# --- UTILITIES (Assumed to be defined in environment) ---
# UNet, compute_ela, rle_encode are assumed to be accessible globals

# --- MAIN EXECUTION: Integrated Confidence Check ---
if __name__ == '__main__':
    
    # 1. Compile list of all 2751 forged files (Same logic as before)
    forged_files = []
    for root, _, files in os.walk(os.path.join(TRAIN_ROOT, 'forged')):
        for f in files:
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
                forged_files.append({
                    'case_id': os.path.splitext(f)[0],
                    'img_path': os.path.join(root, f)
                })
    forged_df = pd.DataFrame(forged_files)

    if forged_df.empty:
        print("🛑 No forged files found for checking.")
        exit()
        
    # 2. Load the Best Model
    try:
        model = UNet(in_channels=4).to(DEVICE)
        model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
        model.eval()
        print(f"Loaded model. Checking {len(forged_df)} forged files for confidence...")
    except Exception as e:
        print(f"🛑 ERROR: Model not loaded. Cannot run confidence check. Error: {e}")
        exit()

    # 3. RUN CONFIDENCE CHECK (MANUAL LOOP)
    low_confidence_cases = []
    
    # Process the files one by one (to simplify input/output handling)
    for index, row in tqdm(forged_df.iterrows(), total=len(forged_df), desc="Checking Confidence"):
        img_path = row['img_path']
        case_id = row['case_id']
        
        try:
            # --- Input Preparation ---
            rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            ela_feature_2d = compute_ela(img_path)
            rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
            stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)
            
            input_tensor = torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32).unsqueeze(0).to(DEVICE)
            
            # --- Prediction ---
            with torch.no_grad():
                output_prob = model(input_tensor).detach().cpu().numpy().squeeze()
            
            max_confidence = np.max(output_prob)
            
            # --- Outlier Flagging ---
            if max_confidence < SUSPICION_THRESHOLD:
                low_confidence_cases.append({
                    'case_id': case_id,
                    'max_confidence': max_confidence,
                    'status': 'Low Confidence'
                })
            
        except Exception as e:
            low_confidence_cases.append({'case_id': case_id, 'max_confidence': -1.0, 'status': f"Error: {e}"})

    # 4. Display Results
    outlier_df = pd.DataFrame(low_confidence_cases).sort_values('max_confidence').reset_index(drop=True)
    
    print(f"\n--- Confidence Check Results (Max Confidence < {SUSPICION_THRESHOLD}) ---")
    print(f"Total files checked: {len(forged_df)}")
    print(f"Total potential outliers found: {len(outlier_df)}")
    
    if not outlier_df.empty:
        print("\nTop 10 Suspicious Cases (Lowest Confidence, Potential Mislabels):")
        print(outlier_df.head(10).to_markdown(index=False))
    else:
        print("\n✅ All forged files had Max Confidence >= 0.40. Dataset appears highly consistent.")

## new trainining

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm # Use tqdm.auto for notebook compatibility
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
from warnings import filterwarnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.optim.lr_scheduler")
filterwarnings('ignore')

# --- CONFIGURATION (FINAL STABLE FIXES) ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
# FINAL FIX 1: Set max epochs to 10
EPOCHS = 10 
# FINAL FIX 2: Reduced LR for stable convergence
LEARNING_RATE = 1e-5 

# --- PATHS (As per original notebook) ---
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"

# --- UTILITY FUNCTIONS (Unchanged) ---
def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE),
                      interpolation=cv2.INTER_LINEAR).astype(np.float32)


# --- LOSS FUNCTIONS (FIXED) ---
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, pred, target):
        pred = pred.contiguous().view(-1)
        target = target.contiguous().view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        inputs = inputs.contiguous().view(-1)
        targets = targets.contiguous().view(-1)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='none')
        BCE_EXP = torch.exp(-BCE)
        Focal = self.alpha * (1-BCE_EXP)**self.gamma * BCE
        return Focal.mean()

class HybridFocalLoss(nn.Module):
    # FINAL FIX 3: Balance loss weights for stable convergence
    def __init__(self, dice_weight=0.5): 
        super(HybridFocalLoss, self).__init__()
        self.dice_loss = DiceLoss() 
        self.focal_loss = FocalLoss()
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        focal = self.focal_loss(pred, target)
        return self.dice_weight * dice + (1 - self.dice_weight) * focal


# U-Net architecture (Unchanged)
class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

class ForgeryDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['img_path']
        mask_path = row['mask_path']

        # --- Load Image (Robust) ---
        rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        if rgb_image is None or rgb_image.size == 0:
            try:
                img_data = np.load(img_path)
                if img_data.ndim == 3: rgb_image = img_data
                elif img_data.ndim == 2: rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)
            except Exception as e:
                raise RuntimeError(f"Failed to load image from {img_path}: {e}")

        # --- Load Mask (Robust) ---
        try:
            mask = np.load(mask_path)
            if mask.ndim > 2: mask = mask[:, :, 0]
        except Exception as e:
            raise RuntimeError(f"Failed to load mask from {mask_path}: {e}")

        ela_feature_2d = compute_ela(img_path)

        # Resize all features
        rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
        ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
        mask_resized = cv2.resize(mask.astype(np.uint8), (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_NEAREST)

        # Stack RGB (3) and ELA (1)
        ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
        stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

        # Convert to PyTorch tensors and normalize
        image = torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
        mask = torch.tensor(mask_resized / 255.0, dtype=torch.float32).unsqueeze(0)

        return image, mask

def train_model(model, train_loader, val_loader, epochs=EPOCHS, save_path=MODEL_SAVE_PATH):
    # Fix 4: Set num_workers=0 to suppress multiprocessing AssertionErrors in notebook environments
    # train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    # val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    criterion = HybridFocalLoss() 
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2) 
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_val_loss = float('inf')
    
    # --- EARLY STOPPING PARAMETERS ---
    PATIENCE = 5                        # Stop if no improvement after 5 epochs
    epochs_no_improve = 0
    MIN_DELTA = 1e-5                    # Minimum improvement required to reset patience (Fix for float comparison)
    # ---------------------------------

    model.to(DEVICE)
    print(f"Starting stabilized training on {DEVICE} for max {epochs} epochs with LR={LEARNING_RATE}...")

    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()

        avg_train_loss = train_loss_sum / len(train_loader)

        # Validation Phase
        model.eval()
        val_loss_sum = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss_sum += loss.item()

        avg_val_loss = val_loss_sum / len(val_loader)

        # Scheduler Step
        scheduler.step(avg_val_loss)

        current_lr = optimizer.param_groups[0]['lr']

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Current LR: {current_lr:.6f}")

        # --- MODEL CHECKPOINTING & EARLY STOPPING LOGIC ---
        if avg_val_loss < best_val_loss - MIN_DELTA:
            # 1. New best loss found: Reset counter and save model
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), save_path)
            print(f"Model saved successfully to {save_path}. (New best Val Loss: {best_val_loss:.4f})\n")
        else:
            # 2. No significant improvement: Increment counter
            epochs_no_improve += 1
            print(f"Validation loss did not significantly improve. Patience left: {PATIENCE - epochs_no_improve}\n")

        # 3. Terminate if patience runs out
        if epochs_no_improve >= PATIENCE:
            print(f"🛑 EARLY STOPPING triggered after {epoch+1} epochs (patience={PATIENCE}). Stopping training.")
            break
            
    print("Training loop finished.")


# --- FUNCTIONS FOR THRESHOLD OPTIMIZATION (Unchanged) ---
def calculate_dice_coefficient(pred, target, smooth=1.0):
    pred = pred.contiguous().view(-1)
    target = target.contiguous().view(-1)
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return dice.item()

def find_optimal_threshold(model, val_loader, device, threshold_range=np.arange(0.05, 0.75, 0.05)):
    model.to(device)
    model.eval()
    
    best_dice = -1.0
    best_threshold = 0.50
    
    print("\n--- Finding Optimal Threshold on Validation Set (Expanded Range) ---")
    
    for threshold in threshold_range:
        total_dice = 0.0
        
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc=f"Evaluating T={threshold:.2f}"):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                predicted_mask = (outputs > threshold).float()
                batch_dice = calculate_dice_coefficient(predicted_mask, targets)
                total_dice += batch_dice

        avg_dice = total_dice / len(val_loader)
        
        if avg_dice > best_dice:
            best_dice = avg_dice
            best_threshold = threshold
            
        # print(f"Threshold: {threshold:.2f} | Avg. Val Dice: {avg_dice:.4f}") # Omitted for cleaner log

    print(f"\n✅ Optimal Threshold found: {best_threshold:.2f} with Dice: {best_dice:.4f}")
    return best_threshold


# --- MAIN EXECUTION BLOCK (Unchanged) ---
if __name__ == '__main__':

    print("Preparing training data paths...\n")

    data_list = []
    # Ensure num_workers=0 is used in DataLoader to prevent multiprocessing errors
    NUM_WORKERS = 0

    # Recursively walk through the TRAIN_ROOT to find all image files
    for root, _, files in os.walk(TRAIN_ROOT):
        for f in files:
            valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')
            if f.lower().endswith(valid_extensions):
                if 'forged' in root.lower():
                    case_id = os.path.splitext(f)[0]
                    img_path = os.path.join(root, f)
                    mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")
                    data_list.append({
                        'case_id': case_id,
                        'img_path': img_path,
                        'mask_path': mask_path
                    })

    if not data_list:
        full_df = pd.DataFrame(columns=['case_id', 'img_path', 'mask_path'])
    else:
        full_df = pd.DataFrame(data_list)

    if not full_df.empty:
        full_df['mask_exists'] = full_df['mask_path'].apply(os.path.exists)
        full_df = full_df[full_df['mask_exists']].drop(columns=['mask_exists']).reset_index(drop=True)

    if full_df.empty:
        print("🛑 FATAL ERROR: No valid image/mask pairs found. Cannot train.")
    else:
        print(f"✅ Found {len(full_df)} valid forged samples for stabilized training.")

        # Split data
        train_df, val_df = train_test_split(full_df, test_size=0.1, random_state=42)

        # Create DataLoaders
        train_dataset = ForgeryDataset(train_df)
        val_dataset = ForgeryDataset(val_df)
        # Use num_workers=0 to prevent multiprocessing AssertionErrors
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        # Instantiate model
        model = UNet(in_channels=4)

        # START STABILIZED TRAINING
        train_model(model, train_loader, val_loader)

        print("\n✅ STABILIZED TRAINING COMPLETE.")

        # --- HYPERPARAMETER TUNING: FIND OPTIMAL THRESHOLD ---
        try:
            model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
            optimal_threshold = find_optimal_threshold(model, val_loader, DEVICE)
            print(f"\n📢 ACTION REQUIRED: Please use {optimal_threshold:.2f} as the FIXED_THRESHOLD in your final inference code.")
            
        except FileNotFoundError:
            print(f"🛑 ERROR: Trained model not found at {MODEL_SAVE_PATH}. Cannot perform threshold tuning.")

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm.auto import tqdm
import time
import csv
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION & PATHS (Final Submission Settings) ---
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8

# CRITICAL: This MUST be manually set to the optimal threshold found
# in your final training run's optimization step (likely ~0.45).
FIXED_THRESHOLD = 0.45 

# CRITICAL: Re-enable the robust area filter (no longer 1)
MIN_FORGERY_AREA = 32

# Paths target the final competition data
TEST_IMAGE_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images"
SAMPLE_SUBMISSION_FILE = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv"
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"
OUTPUT_FILENAME = "submission.csv"

# --- UTILITY FUNCTIONS (Must be in scope) ---
# NOTE: compute_ela, create_test_df_robust, rle_encode, and UNet definitions 
# are assumed to be accessible from previous notebook cells.

def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
    if img is None or img.size == 0: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR).astype(np.float32)

def create_test_df_robust(test_image_root, sample_submission_path):
    master_df = pd.read_csv(sample_submission_path)
    master_df['case_id'] = master_df['case_id'].astype(str)
    present_files = {}
    if os.path.exists(test_image_root):
        for root, _, files in os.walk(test_image_root):
            for f in files:
                case_id = os.path.splitext(f)[0]
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')):
                    present_files[case_id] = os.path.join(root, f)
    master_df['img_path'] = master_df['case_id'].map(present_files)
    master_df['img_path'] = master_df['img_path'].fillna('MISSING_FILE')
    return master_df[['case_id', 'img_path']]

def rle_encode(mask):
    if mask.sum() == 0: return "authentic"
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    # NOTE: Returns space-separated string (e.g., '1456 23 1455 25')
    return ' '.join(str(x) for x in runs)

# U-Net structure must be defined or accessible here
class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )
        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)
    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))


# --- INFERENCE FUNCTION WITH POST-PROCESSING ---
def run_inference_and_segment(unet_model, test_df):
    global FIXED_THRESHOLD, MIN_FORGERY_AREA, TARGET_SIZE, DEVICE
    results = []
    unet_model.eval()
    images_to_process = []
    case_info = []

    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing"):
        case = row['case_id']
        img_path = row['img_path']

        if img_path == 'MISSING_FILE' or img_path == 'NOT_FOUND':
            results.append({'case_id': case, 'annotation': 'authentic'})
            continue

        try:
            # Load Image, ELA, and Preprocess (omitted for brevity, but assumes execution of full loading logic)
            rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0: raise ValueError(f"Invalid image data for {case}")

            original_shape = rgb_image.shape[:2]
            ela_feature_2d = compute_ela(img_path)
            
            # Preprocessing
            rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))
            ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
            stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

            images_to_process.append(
                torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
            )
            case_info.append((case, original_shape))

            if len(images_to_process) == BATCH_SIZE or index == len(test_df) - 1:
                if images_to_process:
                    batch_inputs = torch.stack(images_to_process).to(DEVICE)

                    with torch.no_grad():
                        outputs = unet_model(batch_inputs).detach().cpu().numpy()

                    for i, output in enumerate(outputs):
                        case_id_out, original_shape_out = case_info[i]
                        output_prob = output.squeeze()
                        
                        final_mask_resized = (output_prob > FIXED_THRESHOLD).astype(np.uint8)

                        # --- CRITICAL: Minimum Area Filtering is now active ---
                        clean_mask_resized = np.zeros_like(final_mask_resized)
                        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
                            final_mask_resized, 4, cv2.CV_32S
                        )
                        for label in range(1, num_labels):
                            area = stats[label, cv2.CC_STAT_AREA]
                            if area >= MIN_FORGERY_AREA: # Uses 32
                                clean_mask_resized[labels == label] = 1

                        final_mask = cv2.resize(
                            clean_mask_resized,
                            (original_shape_out[1], original_shape_out[0]),
                            interpolation=cv2.INTER_NEAREST
                        )
                        # --- END CRITICAL ---

                        rle_annotation = rle_encode(final_mask)
                        results.append({'case_id': case_id_out, 'annotation': rle_annotation})

                    images_to_process = []
                    case_info = []

        except Exception as e:
            results.append({'case_id': case, 'annotation': 'authentic'})
    
    return pd.DataFrame(results)

# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":

    print(f"--- Starting Final Submission Inference on {DEVICE} with Threshold={FIXED_THRESHOLD} ---")

    # 1. Load Model (The stable model)
    model = None
    try:
        model = UNet(in_channels=4).to(DEVICE)
        model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
        model.eval()
        print(f"Model loaded successfully from {MODEL_SAVE_PATH}")
    except Exception as e:
        print(f"🛑 ERROR: Model weights not found at {MODEL_SAVE_PATH}. Cannot run inference.")
        model = None

    # 2. Prepare Data (Targets the full Kaggle test set)
    test_df = create_test_df_robust(TEST_IMAGE_ROOT, SAMPLE_SUBMISSION_FILE)

    # 3. Run Inference
    if model:
        results_df = run_inference_and_segment(model, test_df)
    else:
        results_df = test_df[['case_id']].assign(annotation='authentic')

    # 4. Finalize Submission DF and write CSV (Correct RLE Formatting)
    submission_df = test_df[['case_id']].copy().merge(results_df, on='case_id', how='left')
    submission_df['annotation'] = submission_df['annotation'].fillna('authentic')
    submission_df = submission_df[['case_id', 'annotation']].sort_values('case_id').reset_index(drop=True)

    with open(OUTPUT_FILENAME, "w", newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['case_id', 'annotation'])
        for _, row in submission_df.iterrows():
            case_id = str(row['case_id'])
            annotation = row['annotation']
            if annotation.lower() == 'authentic':
                 writer.writerow([case_id, annotation])
            else:
                 # Ensure final RLE string is properly bracketed
                 full_rle_string = f"[{annotation}]" 
                 writer.writerow([case_id, full_rle_string])

    print(f"\n✅ Created final submission file: {OUTPUT_FILENAME}")